In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.3.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [2]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool
import com.google.adk.tools.AgentTool

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [4]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [5]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

---
## 🛣️ Section 4: Parallel Workflows - Independent Researchers

**The Problem: The Bottleneck**

The previous sequential agent is great, but it's an assembly line. Each step must wait for the previous one to finish. What if you have several tasks that are **not dependent** on each other? For example, researching three *different* topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

**The Solution: Concurrent Execution**

When you have independent tasks, you can run them all at the same time using a `ParallelAgent`. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

**Use Parallel when:** Tasks are independent, speed matters, and you can execute concurrently.

To learn more, check out the documentation related to [parallel agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/).

**Architecture: Multi-Topic Research**

```
graph TD
        A["User Request: Research 3 topics"] -- > B["Parallel Execution"]
B -- > C["Tech Researcher"]
B -- > D["Health Researcher"]
B -- > E["Finance Researcher"]

C -- > F["Aggregator"]
D -- > F
E -- > F
F -- > G["Combined Report"]

style B fill:#ffffcc
style F fill:#ffccff
```

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

### 4.1 Example: Parallel Multi-Topic Research

        Let's build a system with four agents:

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
        3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

In [6]:
val techResearcher = LlmAgent.builder()
    .name("tech_researcher")
    .model(geminiModel)
    .description("A technology researcher agent")
    .instruction("""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""")
    .tools(GoogleSearchTool.INSTANCE)
    .outputKey("tech_research")
    .build()

println("✅ Tech researcher agent created.")

✅ Tech researcher agent created.


In [7]:
val healthResearcher = LlmAgent.builder()
    .name("health_researcher")
    .model(geminiModel)
    .description("A health researcher agent")
    .instruction("""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""")
    .tools(GoogleSearchTool.INSTANCE)
    .outputKey("health_research")
    .build()

println("✅ Health researcher agent created.")

✅ Health researcher agent created.


In [8]:
val financeResearcher = LlmAgent.builder()
    .name("finance_researcher")
    .model(geminiModel)
    .description("A finance researcher agent")
    .instruction("""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""")
    .tools(GoogleSearchTool.INSTANCE)
    .outputKey("finance_research")
    .build()

println("✅ Finance researcher agent created.")

✅ Finance researcher agent created.


In [10]:
val aggregatorAgent = LlmAgent.builder()
    .name("aggregator_agent")
    .model(geminiModel)
    .instruction("""
     Combine these three research findings into a single executive summary:
     **Technology Trends:**
     {tech_research}
     **Health Breakthroughs:**
     {health_research}
     **Finance Innovations:**
     {finance_research}

      Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.)""")
    .outputKey("executive_summary")
    .build()

println("✅ Research aggregator agent created.")

✅ Research aggregator agent created.


In [12]:
// The ParallelAgent runs all its sub-agents simultaneously.
val parallelResearchTeam = ParallelAgent.builder()
    .name("parallel_research_team")
    .subAgents(techResearcher, healthResearcher, financeResearcher)
    .build()

println("✅ Parallel research team created")

✅ Parallel research team created


👉 **Then we bring the agents together under a parallel agent, which is itself nested inside of a sequential agent.**

This design ensures that the research agents run first in parallel, then once all of their research is complete, the aggregator agent brings together all of the research findings into a single report:

In [13]:
// The SequentialAgent defines the high level workflow, run the parallel team first, then the aggregator
val rootAgent = SequentialAgent.builder()
    .name("research_system")
    .subAgents(parallelResearchTeam, aggregatorAgent)
    .build()

print("✅ Root agent created")

✅ Root agent created

In [14]:
var researchTeamRunner: InMemoryRunner = InMemoryRunner(rootAgent, "Parallel Research Flow")

In [16]:
var prompt = "Run the daily executive briefing on tech, health, and finance"

In [19]:
var startMsg: Content? = Content.fromParts(Part.fromText(prompt))

In [20]:
var session: Session? = researchTeamRunner
    .sessionService()
    .createSession("Parallel Research Flow", "executive")
    .blockingGet()

In [22]:
var resultText = ""
var events: Flowable<Event> = researchTeamRunner.runAsync("executive", session.id(), startMsg)
println("Pipeline agent response ->")
events.blockingForEach{
    // println(event.stringifyContent())
        event -> event
            .content().get()
            .parts().get()
            .forEach {
                if (it.text().isPresent) {
                    resultText += it.text().get()
                }
            }
}
MIME(
    "text/markdown" to resultText
)

Pipeline agent response ->


**AI/ML Trends: Key Developments, Companies, and Impact**

**1. Generative AI Expansion:** Beyond text, Generative AI (GenAI) now creates graphics, video, and music. Companies like Google (Imagen, Muse) and OpenAI are leading this trend. Its impact lies in revolutionizing content creation across marketing, art, and entertainment.

**2. Agentic AI and Automation:** AI is increasingly acting autonomously, making decisions in areas like logistics and autonomous driving. Companies are exploring agentic AI for enhanced productivity. This trend is transforming operational efficiency and decision-making processes.

**3. Explainable AI (XAI) and Ethics:** As AI adoption grows, transparency in decision-making is crucial, especially in sensitive sectors like healthcare and finance. The focus on XAI aims to build trust and meet regulatory requirements, mitigating bias and ensuring responsible AI usage.

**Companies Involved:** Major players include Google, OpenAI, Microsoft, IBM, and Nvidia.

**Potential Impact:** These trends promise significant advancements in efficiency, creativity, and personalized services across industries. However, they also raise ethical considerations and the need for robust governance and regulatory frameworks.**Medical Breakthroughs on the Horizon**

**1. CRISPR Gene Editing for Genetic Disorders:**
*   **Application:** Precise correction of genetic mutations to treat inherited diseases like sickle cell anemia and cystic fibrosis. Clinical trials are showing promise.
*   **Timeline:** Early-stage clinical applications are already occurring, with broader therapeutic approvals anticipated within the next 3-5 years.

**2. Next-Generation mRNA Vaccines:**
*   **Application:** Beyond COVID-19, mRNA technology is being rapidly developed for influenza, HIV, tuberculosis, and various cancers.
*   **Timeline:** Vaccines for influenza and RSV are in late-stage trials, with broader applications expected within 2-5 years.

**3. AI in Diagnostics and Personalized Medicine:**
*   **Application:** AI algorithms are enhancing diagnostic accuracy for conditions like cancer and heart disease by analyzing medical images and patient data. This allows for tailored treatment plans and predictive health monitoring.
*   **Timeline:** AI is already being integrated into clinical practice, with wider adoption and advanced predictive capabilities expected within the next 1-3 years.Here's an executive briefing on current trends in tech, health, and finance:

**Fintech Trends Report**

Three key fintech trends shaping the market are:

1.  **Embedded Finance:** This trend involves integrating financial services directly into non-financial platforms (e.g., e-commerce checkout). Market implication: Increased customer convenience and new revenue streams for businesses. Future Outlook: Continued growth as more businesses seek to offer seamless financial experiences.

2.  **AI and Machine Learning in Financial Services:** AI is revolutionizing fraud detection, risk assessment, personalized customer service, and algorithmic trading. Market implication: Enhanced efficiency, improved decision-making, and more tailored financial products. Future Outlook: Deeper integration of AI for predictive analytics, hyper-personalization, and automated operations.

3.  **Decentralized Finance (DeFi):** DeFi offers financial services on blockchain technology, bypassing traditional intermediaries. Market implication: Potential for greater financial inclusion, transparency, and reduced costs. Future Outlook: Maturation of the DeFi ecosystem, increased regulatory clarity, and broader adoption, though challenges remain.**Executive Briefing: Daily Trends in Technology, Health, and Finance**

Artificial intelligence (AI) is the overarching theme driving significant advancements across all sectors. In technology, Generative AI is expanding beyond text to create rich media, while agentic AI promises increased automation and efficiency. Explainable AI (XAI) is becoming critical for trust and regulatory compliance, particularly as AI integrates into sensitive areas like finance and healthcare.

This AI integration is profoundly impacting healthcare, with AI-powered diagnostics enhancing accuracy and enabling personalized medicine within 1-3 years. Gene editing (CRISPR) and next-generation mRNA vaccines are also poised for broad therapeutic application within the next 2-5 years, offering novel treatments for genetic disorders and infectious diseases.

Fintech is similarly leveraging AI for enhanced fraud detection, risk assessment, and personalized services. Concurrently, embedded finance is streamlining customer experiences by integrating financial services into non-financial platforms, and Decentralized Finance (DeFi) continues to mature, offering potential for greater financial inclusion.

**Key Takeaways:**

*   **AI is the core enabler:** From content creation to diagnostics and financial services, AI is reshaping industries.
*   **Personalization and efficiency are paramount:** Across all sectors, advancements focus on tailoring services and optimizing operations.
*   **Ethical considerations and regulation are crucial:** As technologies become more powerful and integrated, the need for transparency, trust, and robust governance frameworks is paramount, especially with AI and DeFi.

🎉 Great! You've seen how parallel agents can dramatically speed up workflows by running independent tasks concurrently.

So far, all our workflows run from start to finish and then stop. **But what if you need to review and improve an output multiple times?** Next, we'll build a workflow that can loop and refine its own work.